# This Jupyter Notebook is there to deliver the weekly tweets for CamelQuant

### Loading in libraries and datasets 

In [34]:
# Loading in libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from run_reddit_sentiment import main as run_sentiment

# # # # LOADING IN DATA # # # #
#     -------------------
# 1. BTC Dataset - https://coinmarketcap.com/currencies/bitcoin/historical-data/
btc_stats = pd.read_csv('bitcoin_all.csv', delimiter=';')
columns_to_convert = ['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'timestamp']
btc_stats[columns_to_convert] = btc_stats[columns_to_convert].apply(lambda col: pd.to_datetime(col, utc=True))
btc_stats = btc_stats.sort_values('timestamp').reset_index(drop=True)
btc_stats = btc_stats.drop('name', axis=1)

# 2. Google Trends
gtrends = pd.read_csv('gtrends.csv', header=1)
gtrends.Day = pd.to_datetime(gtrends.Day, utc=True)
gtrends.rename(columns={'bitcoin: (Worldwide)': 'GTrends'}, inplace=True)

# 3. Reddit Sentiment Analysis
reddit = pd.read_csv('BTC_R.csv')
reddit = run_sentiment(reddit)
reddit.date_posted = pd.to_datetime(reddit.date_posted, utc=True)
# 4. Merging the dataframes 
btc_all = pd.merge(btc_stats, gtrends, left_on='timeOpen', right_on='Day', how='left')
btc_all = pd.merge(btc_all, reddit, left_on='timeOpen', right_on='date_posted', how='left')
btc_all = btc_all[['timeOpen', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'GTrends', 'weighted_mean_sentiment']]

KeyError: "['weighted_mean_sentiment'] not in index"

#### Add Technical Indicators to the Dataframe

In [27]:

# 1. Bollinger Bands
def bollinger_bands(df, window=20, num_std=2):
    # Calculate the moving average and standard deviation
    df['MA'] = df['close'].rolling(window=window).mean()
    df['STD'] = df['close'].rolling(window=window).std()
    
    # Calculate upper and lower Bollinger Bands
    df['Upper'] = df['MA'] + (df['STD'] * num_std)
    df['Lower'] = df['MA'] - (df['STD'] * num_std)
    
    # Calculate Bollinger Bands trend determination
    df['bollinger_bands_td'] = np.nan
    df.loc[df['close'] > df['Upper'], 'bollinger_bands_td'] = 0  # Sell signal
    df.loc[df['close'] < df['Lower'], 'bollinger_bands_td'] = 1  # Buy signal
    
    # Forward fill the signals
    df['bollinger_bands_td'] = df['bollinger_bands_td'].ffill()

    df['bollinger_cross'] = df['bollinger_bands_td'].shift(1) == df['bollinger_bands_td']   
    return df
btc_all = bollinger_bands(btc_all)

# 2. MACD
def macd(df, short=12, long=26, signal=9):
    # Calculate the MACD and MACD Signal
    df['MACD'] = df['close'].ewm(span=short).mean() - df['close'].ewm(span=long).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal).mean()
    
    # Calculate MACD trend determination
    df['macd_td'] = np.nan
    df.loc[df['MACD'] > df['MACD_Signal'], 'macd_td'] = 0  # Sell signal
    df.loc[df['MACD'] < df['MACD_Signal'], 'macd_td'] = 1  # Buy signal
    
    # Forward fill the signals
    df['macd_td'] = df['macd_td'].ffill()
    
    return df
btc_all = macd(btc_all)

# 3. RSI
def rsi(df, window=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    def RSI_td(rsi_values):
        if rsi_values >= 70:
            return -1
        elif rsi_values <= 30:
            return 1
        else:
            return 0

    df['RSI_signal'] = df['RSI'].apply(RSI_td)
    return df

btc_all = rsi(btc_all)


# 4. K-Means Support and Resistance Levels
prices = btc_all['close'].values.reshape(-1, 1)
kmeans = KMeans(n_clusters=5)  # Adjust the number of clusters as needed
kmeans.fit(prices) # Get cluster centers (these can be considered as potential support/resistance levels)
cluster_centers = kmeans.cluster_centers_.flatten()
sorted_centers = sorted(cluster_centers, reverse=True) # Sort the levels for better readability
# Create a DataFrame for the support and resistance levels with sorted names
support_resistance_df = pd.DataFrame({
    'Level': sorted_centers,
    'Type': [f'Level {i+1}' for i in range(len(sorted_centers))]
})

# 5. Momentum 

def momentum(df, window):
    df['momentum'] = df.close - df.close.shift(window)
    return df

btc_all = momentum(btc_all, 10)

btc_all

,timeOpen,open,high,low,close,volume,marketCap,GTrends,weighted_sen,MA,...,Upper,Lower,bollinger_bands_td,bollinger_cross,MACD,MACD_Signal,macd_td,RSI,RSI_signal,momentum
0,2023-07-05 00:00:00+00:00,30778.725363,30877.329251,30225.612699,30514.166393,1.248162e+10,5.926152e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,False,0.000000,0.000000,NaN,NaN,0,NaN
1,2023-07-06 00:00:00+00:00,30507.151111,31460.053613,29892.226032,29909.337761,2.112922e+10,5.808949e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,False,-13.569873,-7.538818,1.0,NaN,0,NaN
2,2023-07-07 00:00:00+00:00,29907.998902,30434.643694,29777.284410,30342.264752,1.338477e+10,5.893367e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,False,-3.865672,-6.033430,0.0,NaN,0,NaN
3,2023-07-08 00:00:00+00:00,30346.921653,30374.436525,30080.159610,30292.541173,7.509379e+09,5.884044e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,False,-1.152524,-4.380007,0.0,NaN,0,NaN
4,2023-07-09 00:00:00+00:00,30291.610530,30427.590660,30085.592193,30171.233572,7.903328e+09,5.860798e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,False,-4.481085,-4.410075,1.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2024-06-28 00:00:00+00:00,61612.805785,62126.096568,59985.404260,60320.135526,2.495287e+10,1.189406e+12,24.0,0.005609,64904.943112,...,70642.783653,59167.102570,1.0,True,-1778.983947,-1234.802373,1.0,26.090670,1,-4820.609063
360,2024-06-29 00:00:00+00:00,60319.873202,61097.621824,60300.965102,60887.380835,1.265290e+10,1.200610e+12,21.0,-0.199729,64466.912503,...,70014.604958,58919.220048,1.0,True,-1807.043711,-1349.250641,1.0,28.420330,1,-4072.914466
361,2024-06-30 00:00:00+00:00,60888.445562,62892.827199,60632.950439,62678.292079,1.733323e+10,1.235945e+12,21.0,0.014421,64125.213093,...,69184.825982,59065.600204,1.0,True,-1665.570051,-1412.514523,1.0,35.471869,0,-2150.363732
362,2024-07-01 00:00:00+00:00,62673.606339,63777.227542,62495.510104,62851.980068,2.546838e+10,1.239386e+12,26.0,0.279215,63901.210573,...,68755.560644,59046.860502,1.0,True,-1521.892556,-1434.390129,1.0,36.678955,0,-1244.218120


In [28]:
btc_graph = btc_all.copy()

# Set the style and color palette
plt.style.use('seaborn-v0_8-darkgrid')
desert_colors = {
    'sand': '#E6CCB2',
    'dune': '#C19A6B',
    'cactus': '#7BA05B',
    'sunset': '#FF7F50',
    'sky': '#87CEEB',
    'night': '#1E1E1E',
    'light': '#fffef0'
}

# Create a figure with two subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(40, 25), dpi=300, sharex=True, gridspec_kw={'height_ratios': [3, 1, 1]})
fig.patch.set_facecolor(desert_colors['night'])

# First subplot: Price, Moving Averages, Support/Resistance Levels, and Volume
ax1.plot(btc_graph['timeOpen'], btc_graph['close'], label='Close Price', color=desert_colors['sunset'], linewidth=3)
ax1.plot(btc_graph['timeOpen'], btc_graph['MA'], label='Moving Average (20D)', color=desert_colors['sky'], linewidth=2)
ax1.plot(btc_graph['timeOpen'], btc_graph['Upper'], label='Upper Band', color=desert_colors['cactus'], linewidth=2)
ax1.plot(btc_graph['timeOpen'], btc_graph['Lower'], label='Lower Band', color=desert_colors['cactus'], linewidth=2)
ax1.fill_between(btc_graph['timeOpen'], btc_graph['Upper'], btc_graph['Lower'], color=desert_colors['cactus'], alpha=0.1)
# Adding markers where bollinger_reversal is False
reversal_points = btc_graph[btc_graph['bollinger_cross'] == False]
ax1.scatter(reversal_points['timeOpen'], reversal_points['close'], color='red', label='Reversal Signal', zorder=5)


for i, center in enumerate(sorted_centers):
    ax1.axhline(y=center, color=desert_colors['dune'], linestyle='--', label=f'Level {i+1}: {center:.2f}', linewidth=1.5)

ax1.set_ylabel('Price', fontsize=16, fontweight='bold', color=desert_colors['sand'])
ax1.set_title('BTC Price Analysis', fontsize=24, fontweight='bold', color=desert_colors['light'])
ax1.legend(loc='upper left', fontsize=14, facecolor=desert_colors['light'], edgecolor=desert_colors['light'])

# Volume on the same subplot
ax1_volume = ax1.twinx()
ax1_volume.bar(btc_graph.timeOpen, btc_graph.volume, color=desert_colors['sand'], alpha=0.3, label='Volume')
ax1_volume.set_ylabel('Volume', fontsize=16, fontweight='bold', color=desert_colors['sand'])

# Combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_volume.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left', fontsize=12, facecolor=desert_colors['light'], edgecolor=desert_colors['sand'])

# Second subplot: MACD
ax2.plot(btc_graph['timeOpen'], btc_graph['MACD'], label='MACD', color=desert_colors['sky'], linewidth=2)
ax2.plot(btc_graph['timeOpen'], btc_graph['MACD_Signal'], label='Signal Line', color=desert_colors['sunset'], linewidth=2)
ax2.bar(btc_graph['timeOpen'], btc_graph['MACD'] - btc_graph['MACD_Signal'], label='Histogram', color=desert_colors['sand'], alpha=0.5)

ax2.set_title('MACD Indicator', fontsize=20, fontweight='bold', color=desert_colors['sand'])
ax2.set_xlabel('Date', fontsize=16, fontweight='bold', color=desert_colors['sand'])
ax2.set_ylabel('MACD', fontsize=16, fontweight='bold', color=desert_colors['sand'])
ax2.legend(loc='upper left', fontsize=12, facecolor=desert_colors['night'], edgecolor=desert_colors['sand'])
ax2.grid(True, alpha=0.3, color=desert_colors['sand'])

# Third subplot: RSI
ax3.plot(btc_graph['timeOpen'], btc_graph['RSI'], label='RSI', color=desert_colors['cactus'], linewidth=2)
ax3.axhline(70, linestyle='--', alpha=0.5, color=desert_colors['sunset'])
ax3.axhline(30, linestyle='--', alpha=0.5, color=desert_colors['sky'])
ax3.fill_between(btc_graph['timeOpen'], 30, 70, color=desert_colors['sand'], alpha=0.1)

ax3.set_title('RSI over Time', fontsize=20, fontweight='bold', color=desert_colors['sand'])
ax3.set_xlabel('Date', fontsize=16, fontweight='bold', color=desert_colors['sand'])
ax3.set_ylabel('RSI', fontsize=16, fontweight='bold', color=desert_colors['sand'])
ax3.legend(loc='upper left', fontsize=12, facecolor=desert_colors['night'], edgecolor=desert_colors['sand'])
ax3.grid(True, alpha=0.3, color=desert_colors['sand'])

# Format x-axis
plt.gcf().autofmt_xdate()  # Rotation
myFmt = mdates.DateFormatter('%Y-%m-%d')
ax3.xaxis.set_major_formatter(myFmt)

# Adjust layout
plt.tight_layout()

# Add watermark text
fig.text(0.5, 0.5, 'Camel Quant', fontsize=80, color=desert_colors['light'], 
         ha='center', va='center', alpha=0.4, fontweight='bold', 
         fontfamily='cursive', style='italic')


# Custom styling for all subplots
for ax in [ax1, ax2, ax3]:
    ax.set_facecolor(desert_colors['night'])
    ax.tick_params(colors=desert_colors['sand'], which='both')
    for spine in ax.spines.values():
        spine.set_edgecolor(desert_colors['sand'])

# Show the combined chart
plt.show()

#### Make a fear and greed index

In [36]:

btc_normal_graph = btc_all.copy()
btc_normal_graph = btc_normal_graph[['timeOpen', 'close', 'volume', 'marketCap', 'GTrends', 'weighted_sen', 'RSI', 'momentum']]
btc_normal_graph = btc_normal_graph[btc_normal_graph['timeOpen'] > '2024-02-02']


def fear_and_greed_index(df):
    df['returns'] = df['close'].pct_change()
    df['momentum'] = df['close'].rolling(window=14).mean()
    df['volatility'] = df['returns'].rolling(window=20).std()

    # Define weights for each factor (adjust these based on importance)
    weights = {
        'returns': 0.25,
        'google_trends': 0.15,
        'reddit_sentiment': 0.20,
        'market_cap': 0,
        'momentum': 0.15,
        'volatility': 0.25
    }

    # Initialize Scaler (0-100)
    scaler = MinMaxScaler(feature_range=(0, 100))

    # Normalize inputs using Min-Max scaling
    df['price_change_score'] = scaler.fit_transform(df[['returns']])
    df['google_trends_score'] = df['GTrends']  # Assume it's already 0-100
    df['reddit_sentiment_score'] = (df['weighted_sen'] + 1) * 50  # Convert -1 to 1 range to 0-100
    df['market_cap_score'] = scaler.fit_transform(np.log(df[['marketCap']]))
    df['momentum_score'] = scaler.fit_transform(df[['momentum']])
    df['volatility_score'] = scaler.fit_transform(df[['volatility']])

        # Invert volatility score (high volatility = high fear)
    df['volatility_score'] = 100 - df['volatility_score']

    # Calculate weighted average for Fear and Greed Index
    df['fear_greed_index'] = (
        weights['returns'] * df['price_change_score'] +
        weights['google_trends'] * df['google_trends_score'] +
        weights['reddit_sentiment'] * df['reddit_sentiment_score'] +
        weights['market_cap'] * df['market_cap_score'] +
        weights['momentum'] * df['momentum_score'] +
        weights['volatility'] * df['volatility_score']
    )

    # Assuming df is your DataFrame and 'fear_greed_index' is a numeric column
    df['fear_greed_index'] = df['fear_greed_index'].ewm(span=20, adjust=False).mean()

    return df


btc_normal_graph = fear_and_greed_index(btc_normal_graph)


# Create the figure and axes
fig, ax1 = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor(desert_colors['night'])
ax1.set_facecolor(desert_colors['night'])

# Plot BTC price
ax1.plot(btc_normal_graph['timeOpen'], btc_normal_graph['close'], color=desert_colors['sunset'], linewidth=3, label='BTC Price')
ax1.set_xlabel('Date', fontsize=16, color=desert_colors['sand'])
ax1.set_ylabel('BTC Price', fontsize=16, color=desert_colors['sunset'])
ax1.tick_params(axis='y', labelcolor=desert_colors['sunset'])

# Create twin axis for Fear and Greed Index
ax2 = ax1.twinx()
ax2.plot(btc_normal_graph['timeOpen'], btc_normal_graph['fear_greed_index'], color=desert_colors['cactus'], linewidth=2, label='Fear and Greed Index')
ax2.set_ylabel('Fear and Greed Index', fontsize=16, color=desert_colors['cactus'])
ax2.tick_params(axis='y', labelcolor=desert_colors['cactus'])
ax2.plot(btc_normal_graph['timeOpen'], btc_normal_graph['google_trends_score'], color=desert_colors['sunset'], linewidth=3, label='Google Trends')

# Set title
plt.title('Bitcoin Price vs Fear and Greed Index', fontsize=24, color=desert_colors['sand'])

# Combine legends with improved visibility
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend = ax1.legend(lines1 + lines2, labels1 + labels2, 
                    loc='upper left', 
                    fontsize=14,
                    facecolor=desert_colors['light'],
                    edgecolor=desert_colors['dune'],
                    framealpha=1,  # Fully opaque background
                    fancybox=True,
                    shadow=True,
                    borderpad=1,
                    labelspacing=1.2)

# Ensure legend text is readable
for text in legend.get_texts():
    text.set_color(desert_colors['light'])  # Dark text on light background

# Bring legend to front
ax1.add_artist(legend)

# Format x-axis
plt.gcf().autofmt_xdate()
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Add watermark
fig.text(0.5, 0.5, 'Camel Quant', fontsize=40, color=desert_colors['sand'], 
         ha='center', va='center', alpha=0.3, fontweight='bold', 
         fontfamily='serif', style='italic')

# Custom styling for axes
for ax in [ax1, ax2]:
    ax.tick_params(colors=desert_colors['sand'])
    for spine in ax.spines.values():
        spine.set_edgecolor(desert_colors['sand'])

plt.tight_layout()
plt.show()



KeyError: "['weighted_sen', 'RSI', 'momentum'] not in index"

In [35]:
btc_normal_graph

,timeOpen,close,volume,marketCap,GTrends,weighted_sen,RSI,momentum,returns,volatility,price_change_score,google_trends_score,reddit_sentiment_score,market_cap_score,momentum_score,volatility_score,fear_greed_index
213,2024-02-03 00:00:00+00:00,42992.249629,1.116925e+10,8.433988e+11,23.0,-0.348632,58.399750,NaN,NaN,NaN,NaN,23.0,32.568417,1.753789,NaN,NaN,NaN
214,2024-02-04 00:00:00+00:00,42583.582920,1.480223e+10,8.354218e+11,23.0,0.167146,56.338934,NaN,-0.009506,NaN,40.989422,23.0,58.357296,0.000000,NaN,NaN,NaN
215,2024-02-05 00:00:00+00:00,42658.666048,1.871549e+10,8.369361e+11,25.0,0.060551,75.321455,NaN,0.001763,NaN,47.237410,25.0,53.027549,0.334210,NaN,NaN,NaN
216,2024-02-06 00:00:00+00:00,43084.671864,1.679848e+10,8.453385e+11,24.0,-0.083631,75.664913,NaN,0.009986,NaN,51.796765,24.0,45.818444,2.177726,NaN,NaN,NaN
217,2024-02-07 00:00:00+00:00,44318.224451,2.112659e+10,8.695853e+11,24.0,0.114028,78.999574,NaN,0.028631,NaN,62.134230,24.0,55.701396,7.396567,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2024-06-28 00:00:00+00:00,60320.135526,2.495287e+10,1.189406e+12,24.0,0.005609,26.090670,63614.134542,-0.020853,0.016832,34.697649,24.0,50.280449,65.195222,74.715462,87.738255,61.390003
360,2024-06-29 00:00:00+00:00,60887.380835,1.265290e+10,1.200610e+12,21.0,-0.199729,28.420330,63235.304568,0.009404,0.017024,51.473814,21.0,40.013575,66.925434,72.998705,87.172263,60.949435
361,2024-06-30 00:00:00+00:00,62678.292079,1.733323e+10,1.235945e+12,21.0,0.014421,35.471869,62952.393536,0.029414,0.018821,62.568150,21.0,50.721045,72.278360,71.716628,81.881663,60.874647
362,2024-07-01 00:00:00+00:00,62851.980068,2.546838e+10,1.239386e+12,26.0,0.279215,36.678955,62692.513956,0.002771,0.017825,47.796246,26.0,63.960773,72.791539,70.538923,84.814884,60.831897
